# Running Our Trained Transfer Learning Example on a Single Image

In [1]:
import os
from urllib import request
from zipfile import ZipFile

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, models, transforms

from skorch import NeuralNetClassifier
from skorch.helper import predefined_split

torch.manual_seed(360)

## Example Code to Classify a Single Image

In [4]:
# define ImageNet-based transformations (normalized with mean and standard deviation of ImageNet images)

model_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# define pretrained ResNet18 neural network model with final layer replaced with a fully connected
# layer (in using this, we will want to make the number of output features 2 since we trained 
# a binary ant/bee classifer)
class PretrainedModel(nn.Module):
    def __init__(self, output_features):
        super().__init__()
        model = models.resnet18(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, output_features)
        self.model = model
        
    def forward(self, x):
        return self.model(x)

# create skorch.NeuralNetClassifier as in tutorial

#from skorch.callbacks import LRScheduler
#lrscheduler = LRScheduler(
#    policy='StepLR', step_size=7, gamma=0.1)
#from skorch.callbacks import Checkpoint
#checkpoint = Checkpoint(
#    f_params='best_model.pt', monitor='valid_acc_best')
#from skorch.callbacks import Freezer
#freezer = Freezer(lambda x: not x.startswith('model.fc'))

net = NeuralNetClassifier(
    PretrainedModel, 
    criterion=nn.CrossEntropyLoss,
    #lr=0.001,
    #batch_size=4,
    #max_epochs=25,
    module__output_features=2,
    #optimizer=optim.SGD,
    #optimizer__momentum=0.9,
    #iterator_train__shuffle=True,
    #iterator_train__num_workers=2,
    #iterator_valid__num_workers=2,
    #train_split=predefined_split(val_ds),
    #callbacks=[lrscheduler, checkpoint, freezer],
    #device='cuda' # comment to train on cpu
)

# LOAD MODEL FROM FILE
net.initialize()
net.load_params(f_params='best_model.pt')

# APPLY TO IMAGE
from PIL import Image
#image_path = 'antImage.jpg'
image_path = 'beeImage.jpg'

# load image
img = Image.open(image_path)

# apply transformations
img_transformed = model_transforms(img).unsqueeze(0)

# apply model
output = net.predict(img_transformed)
print(output) # 0=ant, 1=bee

[1]
